In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
from gensim.models import FastText
import sys
from gensim.test.utils import datapath

sys.path.append('..')

from emb_utils import DimenGeneratorGensim

def load_fasttext_model(lang_code: str, data_source: str = 'synth-bedtime') -> FastText:
    """Load the FastText model for the given language code"""
    model_path = f"../embeddings/{data_source}/Meta-Llama-3-1-8B-Instruct-nwxcg_{lang_code}/fasttext.model"
    model = FastText.load(model_path)
    return model

emb = load_fasttext_model('en')

In [ ]:
import pickle
def load_training_data(lang_code: str) -> list:
    """Load the training data for the given language code"""
    training_data_path = f"../data/wiki_gen_cleaned/Meta-Llama-3-1-8B-Instruct-nwxcg/{lang_code}/processed_articles.pkl"
    with open(training_data_path, "rb") as f:
        training_data = pickle.load(f)
    return training_data

from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

def load_and_save_glove()
glove_input_file = '/scratch/gpfs/vv7118/projects/GloVe/embs/en/vectors.txt'
word2vec_output_file = '/scratch/gpfs/vv7118/projects/GloVe/embs/en/vectors_word2vec.txt'

# Convert GloVe to Word2Vec format
glove2word2vec(glove_input_file, word2vec_output_file)

# Load the converted file
glove = KeyedVectors.load_word2vec_format(word2vec_output_file)

/tmp/ipykernel_2184441/2477258201.py:8: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file, word2vec_output_file)


In [6]:
embgt =  KeyedVectors.load_word2vec_format('/scratch/gpfs/vv7118/projects/semantic-accents/data/embs/wiki.en.vec', limit=50000)


In [5]:
glove.evaluate_word_pairs(datapath('wordsim353.tsv'))

(PearsonRResult(statistic=0.43736440526800957, pvalue=7.510387020358823e-17),
 SignificanceResult(statistic=0.4441452925736125, pvalue=2.191712062858109e-17),
 6.515580736543909)

In [19]:
analogies = glove.evaluate_word_analogies(datapath('questions-words.txt'))

In [7]:
analogies = glove.most_similar(positive=['female', 'king'], negative=['male'], topn=10)


In [15]:
glove.most_similar('vaccination')

NameError: name 'glove' is not defined

In [ ]:
seeds = DimenGeneratorGensim(glove)


In [21]:
selected_seeds = seeds.generate_dimension_from_seeds([('twitter', 'facebook')])

seed directions...
scoring directions...
ranking directions...
selecting pairs...


In [22]:
selected_seeds['left_comms']

['variance',
 'threads',
 'symbolize',
 'disrupting',
 'top10',
 'twitters',
 'lydia',
 'wilhelms',
 'pinkett',
 'songwriting']

In [23]:
selected_seeds['right_comms']

['tva',
 'modular',
 'decriminalization',
 'chica',
 'bertolt',
 '4chan',
 'kaz',
 'robbies',
 'galante',
 'healys']

In [55]:
#embgt.most_similar('reddit')
embgt.evaluate_word_pairs(datapath('wordsim353.tsv'))

(PearsonRResult(statistic=0.6987675548794924, pvalue=5.2375793741607e-53),
 SignificanceResult(statistic=0.7388081960366618, pvalue=3.98104844873057e-62),
 0.0)

In [2]:
emb.wv.evaluate_word_pairs(datapath('wordsim353.tsv'))

(PearsonRResult(statistic=0.48340347994626637, pvalue=1.0797489198683427e-15),
 SignificanceResult(statistic=0.4583488054761287, pvalue=4.4591104218673675e-14),
 30.878186968838527)

In [3]:
emb.wv.most_similar('nationalism')

[('factionalism', 0.9075059294700623),
 ('regionalism', 0.8905136585235596),
 ('internationalism', 0.8481733798980713),
 ('constitutionalism', 0.8108145594596863),
 ('paternalism', 0.8078532218933105),
 ('conventionalism', 0.795631468296051),
 ('liberalism', 0.7798908352851868),
 ('congregationalism', 0.7782041430473328),
 ('inequality', 0.7645931839942932),
 ('national', 0.7480834126472473)]

In [4]:
df = load_training_data("ru")

In [5]:
import pandas as pd 
from glob import glob 

files = glob('/scratch/gpfs/vv7118/projects/semantic-accents/data/wiki_gen_cleaned/Meta-Llama-3-1-8B-Instruct-nwxcg/ru/*.jsonl')

raw_data = pd.DataFrame()
for f in files:
    raw_data = pd.concat([raw_data, pd.read_json(f, lines=True)])

In [6]:
import random
from IPython.display import display, HTML

def print_word_segments(df, word):
    """
    Display metadata and 10 random text segments from the 'article' column of a DataFrame
    where a specified word is present. Each segment includes 10 words
    before and 10 words after the word, with the word highlighted in bold and color.
    
    Args:
        df (pd.DataFrame): DataFrame containing an 'article' column.
        word (str): The word to search for in the articles.
    """
    # Filter rows where the word exists in the 'article' column
    matching_rows = df[df['article'].str.contains(rf'\b{word}\b', case=False, na=False)]
    total_rows = len(df)
    word_occurrence = len(matching_rows)
    
    if matching_rows.empty:
        display(HTML(f"<p style='color:red;'>No rows found containing the word <b>{word}</b>.</p>"))
        return

    # Calculate metadata
    fraction = word_occurrence / total_rows

    # Display metadata
    metadata_html = f"""
    <h3 style='color:blue;'>Metadata</h3>
    <ul>
        <li>Number of rows containing the word <b>{word}</b>: <b>{word_occurrence}</b></li>
        <li>Total rows in DataFrame: <b>{total_rows}</b></li>
        <li>Fraction of rows containing the word: <b>{fraction:.4f}</b></li>
    </ul>
    """
    display(HTML(metadata_html))

    # Randomly select up to 10 rows
    selected_rows = matching_rows.sample(n=min(10, word_occurrence))

    html_output = ""
    for idx, row in selected_rows.iterrows():
        article = row['article']
        words = article.split()
        segments = []
        
        for i, w in enumerate(words):
            if w.lower() == word.lower():
                # Extract 10 words before and after
                start = max(0, i - 10)
                end = min(len(words), i + 11)
                # Highlight the target word in bold and cyan
                context = ' '.join(
                    f"<span style='color:cyan; font-weight:bold;'>{words[j]}</span>" if j == i else words[j]
                    for j in range(start, end)
                )
                segments.append(context)
        
        html_output += f"<h4 style='color:orange;'>--- Row Index {idx} ---</h4>"
        for segment in segments:
            html_output += f"<p style='color:green;'>... {segment} ...</p>"

    display(HTML(html_output))

In [12]:
print_word_segments(raw_data, 'ссылаться')

In [17]:
emb.wv.most_similar('twitter')

[('instagram', 0.9168775677680969),
 ('dast', 0.9007313847541809),
 ('placebo', 0.8838208913803101),
 ('google', 0.8830086588859558),
 ('facebook', 0.870422899723053),
 ('twitch', 0.8600685000419617),
 ('баны', 0.8406918048858643),
 ('face', 0.8326565623283386),
 ('reddit', 0.8284628987312317),
 ('youtubecom', 0.8248284459114075)]

In [ ]:

emb.wv.evaluate_word_pairs(datapath('wordsim353.tsv'))

(PearsonRResult(statistic=0.5039531358727269, pvalue=2.3055736711007882e-23),
 SignificanceResult(statistic=0.5129955391332383, pvalue=2.7683743776975242e-24),
 3.39943342776204)

In [32]:
'combine' in emb.wv.key_to_index

True

In [8]:
word_analogies = emb.wv.evaluate_word_analogies(datapath('questions-words.txt'))


In [16]:
word_analogies[1][5]


{'section': 'gram1-adjective-to-adverb',
 'correct': [('COMPLETE', 'COMPLETELY', 'EFFICIENT', 'EFFICIENTLY'),
  ('COMPLETE', 'COMPLETELY', 'OCCASIONAL', 'OCCASIONALLY'),
  ('COMPLETE', 'COMPLETELY', 'POSSIBLE', 'POSSIBLY'),
  ('COMPLETE', 'COMPLETELY', 'QUIET', 'QUIETLY'),
  ('COMPLETE', 'COMPLETELY', 'SERIOUS', 'SERIOUSLY'),
  ('COMPLETE', 'COMPLETELY', 'SUDDEN', 'SUDDENLY'),
  ('COMPLETE', 'COMPLETELY', 'TYPICAL', 'TYPICALLY'),
  ('EFFICIENT', 'EFFICIENTLY', 'FREE', 'FREELY'),
  ('EFFICIENT', 'EFFICIENTLY', 'OCCASIONAL', 'OCCASIONALLY'),
  ('EFFICIENT', 'EFFICIENTLY', 'POSSIBLE', 'POSSIBLY'),
  ('EFFICIENT', 'EFFICIENTLY', 'QUIET', 'QUIETLY'),
  ('EFFICIENT', 'EFFICIENTLY', 'RAPID', 'RAPIDLY'),
  ('EFFICIENT', 'EFFICIENTLY', 'SERIOUS', 'SERIOUSLY'),
  ('EFFICIENT', 'EFFICIENTLY', 'SUDDEN', 'SUDDENLY'),
  ('EFFICIENT', 'EFFICIENTLY', 'TYPICAL', 'TYPICALLY'),
  ('EFFICIENT', 'EFFICIENTLY', 'USUAL', 'USUALLY'),
  ('EFFICIENT', 'EFFICIENTLY', 'COMPLETE', 'COMPLETELY'),
  ('FREE', 'FREELY